In [25]:
import requests, zipfile, re, os, fiona
import geopandas as gpd
import pandas as pd

pd.set_option('display.max_columns', None)

In [26]:
extract_location = r'../data_download'

In [ ]:
response = requests.get('https://disasters.geoplatform.gov/USA_Structures')
print(response)

In [ ]:
files = re.findall(pattern=r'USA_Structures/\w+\+*\w*/Deliverable........\S\S.zip', string=str(response._content))
print(files)

In [ ]:
def download_and_extract_zip(url, extract_to=r'.', filename=None):
    if filename is None:
        filename = url.split('/')[-1] # retrieve final portion of URL for file name
    
    with requests.get(url, stream=True) as r:
        r.raise_for_status()

        with open(filename, "wb") as f:
            for chunk in r.iter_content(chunk_size=1024):
                f.write(chunk)
    
    with zipfile.ZipFile(filename, 'r') as z:
        z.extractall(path=extract_to)

In [ ]:
for file in files:
    print(f'Processing {file}...')
    download_and_extract_zip(
        url=f'https://fema-femadata.s3.amazonaws.com/Partners/ORNL/{file}',
        extract_to=extract_location
    )

In [ ]:
gdbs = []

for root, dir, file in os.walk(extract_location):
    if len(dir) > 0:
        for i in list(range(0, len(dir))):
            if dir[i][-4:] == '.gdb':
                gdbs.append(fr'{root}/{dir[i]}')
print(gdbs)

In [ ]:
gdf = gpd.GeoDataFrame()

for gdb in gdbs:
    layers = fiona.listlayers(gdb)

    for layer in layers:
        print(f'Reading {layer} in {gdb}...')
        gdf = gpd.read_file(filename=gdb, driver='OpenFileGDB', layer=layer)
        print(f'  > {layer} has {len(gdf.index)} total rows.')

        gdf = gdf.dropna(subset=['PROP_ADDR', 'PROP_CITY']).reset_index(drop=True)
        print(f' > {layer} has {len(gdf.index)} rows with address information.')

        gdf.to_parquet(path=fr'./processing/{layer}.parquet', index=False)
        """
        rows_to_add = len(temp.index)
        expected_rows = len(gdf.index) + len(temp.index)

        gdf = pd.concat([gdf, temp], ignore_index=True)
        print(f'   > Full GDF now has {len(gdf.index)} total rows.')

        if len(gdf.index) != expected_rows:
            #raise ValueException
            print(f'WARNING: EXPECTED ROW MISMATCH. Expected {expected_rows} rows, but returned {len(gdf.index)}.')
        """

In [32]:
parqs = [dir for dir in os.listdir(r'./processing') if dir.endswith('.parquet')]

del gdf
gdf = gpd.GeoDataFrame()

for parq in parqs:
    print(f'Merging {parq}...')
    gdf = pd.concat(objs=[gdf, gpd.read_parquet(fr'./processing/{parq}')], ignore_index=True)
gdf.to_parquet(r'./outputs/merged_20251025.parquet', index=False)

Merging AK_Structures.parquet...
Merging AL_Structures.parquet...
Merging AR_Structures.parquet...
Merging AS_Structures.parquet...
Merging AZ_Structures.parquet...
Merging CA_Structures.parquet...
Merging CO_Structures.parquet...
Merging CT_Structures.parquet...
Merging DE_Structures.parquet...
Merging FL_Structures.parquet...
Merging GA_Structures.parquet...
Merging GU_Structures.parquet...
Merging HI_Structures.parquet...
Merging IA_Structures.parquet...
Merging ID_Structures.parquet...
Merging IL_Structures.parquet...
Merging IN_Structures.parquet...
Merging KS_Structures.parquet...
Merging KY_Structures.parquet...
Merging LA_Structures.parquet...
Merging MA_Structures.parquet...
Merging MD_Structures.parquet...
Merging ME_Structures.parquet...
Merging MI_Structures.parquet...
Merging MN_Structures.parquet...
Merging MO_Structures.parquet...
Merging MS_Structures.parquet...
Merging MT_Structures.parquet...
Merging NC_Structures.parquet...
Merging ND_Structures.parquet...
Merging NE

MemoryError: Unable to allocate 950. MiB for an array with shape (3, 41509844) and data type object